In [12]:
from pathlib import Path
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
api_key  = os.getenv('OPENAI_APIKEY')
vector_store_id=os.getenv('VECTOR_STORE_ID')
assistant_id= os.getenv('ASSISTANT_ID')

In [13]:

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

In [14]:
thread = client.beta.threads.create()

In [9]:
prompt = "緊急避難の方法を教えてください"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

In [10]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant_id
)

In [11]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")

    # file_citationがある場合の処理
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

緊急避難の方法についての情報は以下の通りです。

1. **屋外への避難時**：
   - 常にエレベーターは使わず、階段を使用すること。
   - 屋上への避難はしないように。
   - 慌てずに周囲の様子を確認し、足元の散乱物や頭上の落下物に注意する。
   - 押し合ったりせず、周囲の人の安全にも気を配る。
   - 火災が発生している場合は、タオルやハンカチで口を覆い、煙からの安全を確保する[0]。

2. **避難後の確認**：
   - 避難後は迅速に人員点検を行い、逃げ遅れた人がいないかを確認する。
   - 忘れ物があっても再び建物には戻らないようにする。
   - 災害の状況を冷静に把握し、パニックを防止すること[0]。

この情報は、地震発生時に安全に避難するための重要な指針を提供しており、集団での行動時にも心がけるべきポイントです。
[0] manual_jishin.pdf
[1] manual_jishin.pdf
